# การเตรียมข้อมูลและสร้างไฟล์ Submission สำหรับ Kaggle

โน้ตบุ๊กนี้จะทำการโหลดข้อมูล Train และ Test, ทำการจัดการข้อมูล (Preprocessing), เทรนโมเดล (Decision Tree และ Random Forest), และสร้างไฟล์ Submission (`submission_dt.csv`, `submission_rf.csv`)

In [ ]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# โหลดข้อมูล
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# [FIX] จัดการปัญหากรณีชื่อคอลัมน์มีวรรคเกิน (Whitespace fix)
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# [FIX] จัดการปัญหาข้อมูล string มีวรรคเกิน
def clean_strings(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).str.strip()
    return df

train_df = clean_strings(train_df)
test_df = clean_strings(test_df)

print("Train shape:", train_df.shape)
print("Train columns:", train_df.columns.tolist())

Train shape: (891, 12)
Train columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


## 1. การจัดการข้อมูล (Data Preprocessing)

เราจะทำการ:
1. เติมข้อมูลที่หายไป (`Age`, `Embarked`, `Fare`)
2. ลบคอลัมน์ที่ไม่จำเป็น (`Cabin`, `Name`, `Ticket`, `PassengerId`)
3. แปลงข้อมูลตัวอักษรเป็นตัวเลข (`Sex`, `Embarked`)

In [ ]:
# เก็บ PassengerId ไว้สำหรับไฟล์ submission ก่อนที่จะลบทิ้ง
test_passenger_ids = test_df['PassengerId']

# Preprocessing Function
def preprocess_data(df, is_train=True):
    df = df.copy()
    
    # แปลงตัวเลขเป็น numeric (เผื่อกรณีติด string)
    cols = ['Age', 'Fare', 'SibSp', 'Parch']
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
            
    # เติมข้อมูลที่หายไป
    if 'Age' in df.columns:
        df['Age'].fillna(df['Age'].median(), inplace=True)
    else:
        print("Warning: Age column not found")

    if 'Embarked' in df.columns:
        df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
        
    if 'Fare' in df.columns:
        df['Fare'].fillna(df['Fare'].median(), inplace=True)
    
    # ลบคอลัมน์ที่ไม่ใช้
    cols_to_drop = ['Cabin', 'Name', 'Ticket', 'PassengerId']
    df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    
    # แปลงข้อมูลเป็นตัวเลข
    sex_map = {'male': 0, 'female': 1}
    embarked_map = {'S': 0, 'C': 1, 'Q': 2}
    
    if 'Sex' in df.columns:
        df['Sex'] = df['Sex'].map(sex_map)
        
    if 'Embarked' in df.columns:
        df['Embarked'] = df['Embarked'].map(embarked_map)
    
    return df

# Apply Preprocessing
train_processed = preprocess_data(train_df)
test_processed = preprocess_data(test_df, is_train=False)

# ตรวจสอบว่ามีค่า Null เหลือหรือไม่
print("Nulls in Train:", train_processed.isnull().sum().sum())

# เตรียม Features และ Target
if 'Survived' in train_processed.columns:
    X_train = train_processed.drop('Survived', axis=1)
    y_train = train_processed['Survived']
else:
    print("Error: Survived column missing from training data")
    
X_test = test_processed

print("Features:", X_train.columns.tolist())

Nulls in Train: 2
Features: ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']


## 2. การเทรนโมเดล (Model Training)

### 2.1 Decision Tree

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

dt_pred = dt_model.predict(X_test)
print("Decision Tree Trained.")

Decision Tree Trained.


### 2.2 Random Forest

ใช้ `n_estimators=100` และกำหนด `max_depth=5` ตามที่ต้องการ

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict(X_test)
print("Random Forest Trained.")

Random Forest Trained.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)
print("Gradient Boosting Trained.")

### 2.4 XGBoost (Extreme Gradient Boosting)

โมเดลยอดนิยมที่ปรับปรุงประสิทธิภาพมาจาก Gradient Boosting

In [ ]:
from xgboost import XGBClassifier
# สร้างและเทรนโมเดล XGBoost
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# ทำนายผล
xgb_pred = xgb_model.predict(X_test)
print("XGBoost Trained.")

## 3. สร้างไฟล์ Submission

In [ ]:
submission_dt = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': dt_pred
})
submission_dt.to_csv('submission_dt.csv', index=False)
print("Saved submission_dt.csv")

submission_rf = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': rf_pred
})
submission_rf.to_csv('submission_rf.csv', index=False)
print("Saved submission_rf.csv")

# Submission สำหรับ XGBoost
submission_xgb = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': xgb_pred
})
submission_xgb.to_csv('submission_xgb.csv', index=False)
print("Saved submission_xgb.csv")

Saved submission_dt.csv
Saved submission_rf.csv
